In [1]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=37902401505f28a4524aecc4ef86a5ec01dd39770177cb3e07eaf94c694e0e6f
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [22]:
!pip install duckdb

In [2]:
from pandasql import sqldf
import pandas as pd

In [23]:
import duckdb

In [7]:
from google.colab import drive
drive.mount('mydrive/')

Mounted at mydrive/


In [10]:
file_path='/content/mydrive/MyDrive/employee_database.xlsx'

In [13]:
# Read all sheets into a dictionary of DataFrames
all_sheets = pd.read_excel(file_path, sheet_name=None)
data={}
for sheet_name, sheet_df in all_sheets.items():
    print(f"Sheet name: {sheet_name}")
    data[sheet_name]=sheet_df


Sheet name: employee
Sheet name: department
Sheet name: location


In [14]:
data['employee'].head()

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,HIRE_DATE,JOB_ID,SALARY,DEPARTMENT_ID,RESIDENCY_ID
0,198,Donald,OConnell,DOCONNEL,2007-06-21,SH_CLERK,2600,50,1500
1,199,Douglas,Grant,DGRANT,2008-01-13,SH_CLERK,2600,50,1500
2,200,Jennifer,Whalen,JWHALEN,2003-09-17,AD_ASST,4400,10,1700
3,201,Michael,Hartstein,MHARTSTE,2004-02-17,MK_MAN,13000,20,1000
4,202,Pat,Fay,PFAY,2005-08-17,MK_REP,6000,20,1800


In [15]:
data['employee'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   EMPLOYEE_ID    50 non-null     int64         
 1   FIRST_NAME     50 non-null     object        
 2   LAST_NAME      50 non-null     object        
 3   EMAIL          50 non-null     object        
 4   HIRE_DATE      50 non-null     datetime64[ns]
 5   JOB_ID         50 non-null     object        
 6   SALARY         50 non-null     int64         
 7   DEPARTMENT_ID  50 non-null     int64         
 8   RESIDENCY_ID   50 non-null     int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 3.6+ KB


In [29]:
employee_df=data['employee']

In [16]:
data['department'].head()

,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID
0,10,Administration,200.0,1700
1,20,Marketing,201.0,1800
2,30,Purchasing,114.0,1700
3,40,Human Resources,203.0,2400
4,50,Shipping,121.0,2000


In [17]:
data['department'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DEPARTMENT_ID    12 non-null     int64  
 1   DEPARTMENT_NAME  12 non-null     object 
 2   MANAGER_ID       11 non-null     float64
 3   LOCATION_ID      12 non-null     int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 512.0+ bytes


In [30]:
department_df=data['department']

In [19]:
data['location'].head()

,LOCATION_ID,LOCATION_NAME,STATE,COUNTRY
0,1700,Ikeja,Lagos,Nigeria
1,1800,Ibadan,Oyo,Nigeria
2,1900,Kaduna,Kaduna,Nigeria
3,2000,PH,Rivers,Nigeria
4,2400,Warri,Delta,Nigeria


In [20]:
data['location'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   LOCATION_ID    12 non-null     int64 
 1   LOCATION_NAME  12 non-null     object
 2   STATE          12 non-null     object
 3   COUNTRY        12 non-null     object
dtypes: int64(1), object(3)
memory usage: 512.0+ bytes


In [31]:
location_df=data['location']

###Qst 1
Number of remote workers in each department i.e. staff working outside their department location excluding shipping staff?


In [34]:
sqldf('select distinct JOB_ID from employee_df')

ObjectNotExecutableError: ignored

In [38]:
duckdb.sql("select distinct job_id from employee_df").df()

,JOB_ID
0,SH_CLERK
1,AD_ASST
2,MK_MAN
3,MK_REP
4,HR_REP
5,PR_REP
6,AC_MGR
7,AC_ACCOUNT
8,AD_PRES
9,AD_VP


In [39]:
duckdb.sql("select distinct department_name from department_df").df()

,DEPARTMENT_NAME
0,Administration
1,Marketing
2,Purchasing
3,Human Resources
4,Shipping
5,IT
6,Public Relations
7,Sales
8,Executive
9,Finance


In [47]:
query="""
  SELECT
    department_name AS department, count(*) remote_worker
  FROM
    (
      SELECT
        em.*,
        de.department_name,
        de.location_id

      FROM employee_df em
      JOIN department_df de
        ON em.department_id = de.department_id
      WHERE
        de.location_id <> em.residency_id
        AND lower(de.department_name) <> 'shipping'
    )
  GROUP BY 1
"""
duckdb.sql(query).df()

,department,remote_worker
0,Marketing,1
1,Senior_Executive,1
2,IT,3
3,Purchasing,4


### Qst 2
Create a manager information dimension table


In [53]:
query_2 = """
  SELECT
    em.*,
    de.department_name,
    de.location_id AS department_location_id
  FROM department_df de
  JOIN employee_df em
    ON em.employee_id= de.manager_id
"""
manager_df=duckdb.sql(query_2).df()
manager_df

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,HIRE_DATE,JOB_ID,SALARY,DEPARTMENT_ID,RESIDENCY_ID,DEPARTMENT_NAME,department_location_id
0,200,Jennifer,Whalen,JWHALEN,2003-09-17,AD_ASST,4400,10,1700,Administration,1700
1,201,Michael,Hartstein,MHARTSTE,2004-02-17,MK_MAN,13000,20,1000,Marketing,1800
2,203,Susan,Mavris,SMAVRIS,2002-06-07,HR_REP,6500,40,2400,Human Resources,2400
3,204,Hermann,Baer,HBAER,2002-06-07,PR_REP,10000,70,2700,Public Relations,2700
4,205,Shelley,Higgins,SHIGGINS,2002-06-07,AC_MGR,12008,110,1700,Accounting,1700
5,100,Steven,King,SKING,2003-06-17,AD_PRES,24000,0,3000,Executive,1700
6,103,Alexander,Hunold,AHUNOLD,2006-01-03,IT_PROG,9000,60,1700,IT,1400
7,108,Nancy,Greenberg,NGREENBE,2002-08-17,FI_MGR,12008,100,1700,Finance,1700
8,114,Den,Raphaely,DRAPHEAL,2002-12-07,PU_MAN,11000,30,1800,Purchasing,1700
9,121,Adam,Fripp,AFRIPP,2005-04-10,ST_MAN,8200,50,1900,Shipping,2000


### Qst 3
Number of workers earning above the average salary in the organization


In [54]:
query_3 = """
  SELECT
    count(*) high_earned_workers
  FROM employee_df
  WHERE salary >
    (SELECT AVG(salary) FROM employee_df)
"""
duckdb.sql(query_3).df()

,high_earned_workers
0,20


### Qst 4
Create a new column in the employee table categorizing staff as either early hire or late based on the hire date i.e. if the hire date is greater or equal to the average hire date in the organization that should be late hire else early hire as this will assist in shares distribution


In [95]:
query_4 = """
  SELECT
    ROUND(date_diff('day', MIN(hire_date), MAX(hire_date))/2) AS days,
    MIN(hire_date) minimum_hire_date,
    MAX(hire_date) maximum_hire_date,
  FROM employee_df
"""
duckdb.sql(query_4).df()


,days,min(hire_date),max(hire_date)
0,1306.0,2001-01-13,2008-03-08


In [97]:
avg_hire_date = employee_df['HIRE_DATE'].mean()

In [100]:
avg_hire_date.date()

2005-06-03


In [106]:
query_4 = f"""
  SELECT  *,
  CASE
    WHEN hire_date >= '{avg_hire_date.date()}' THEN 'LATE HIRED'
    ELSE 'EARLY HIRED'
  END AS hire_type
  FROM employee_df

"""
employee_extended=duckdb.sql(query_4).df()
employee_extended

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,HIRE_DATE,JOB_ID,SALARY,DEPARTMENT_ID,RESIDENCY_ID,hire_type
0,198,Donald,OConnell,DOCONNEL,2007-06-21,SH_CLERK,2600,50,1500,LATE HIRED
1,199,Douglas,Grant,DGRANT,2008-01-13,SH_CLERK,2600,50,1500,LATE HIRED
2,200,Jennifer,Whalen,JWHALEN,2003-09-17,AD_ASST,4400,10,1700,EARLY HIRED
3,201,Michael,Hartstein,MHARTSTE,2004-02-17,MK_MAN,13000,20,1000,EARLY HIRED
4,202,Pat,Fay,PFAY,2005-08-17,MK_REP,6000,20,1800,LATE HIRED
5,203,Susan,Mavris,SMAVRIS,2002-06-07,HR_REP,6500,40,2400,EARLY HIRED
6,204,Hermann,Baer,HBAER,2002-06-07,PR_REP,10000,70,2700,EARLY HIRED
7,205,Shelley,Higgins,SHIGGINS,2002-06-07,AC_MGR,12008,110,1700,EARLY HIRED
8,206,William,Gietz,WGIETZ,2002-06-07,AC_ACCOUNT,8300,110,1700,EARLY HIRED
9,100,Steven,King,SKING,2003-06-17,AD_PRES,24000,0,3000,EARLY HIRED
